In [1]:
import os
from dotenv import load_dotenv
from datetime import datetime

import openai
from IPython.display import HTML, display
from ipywidgets import widgets
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma

load_dotenv()

True

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter


text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 600,
    chunk_overlap = 100,
    length_function = len,
)

openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
print(openai.api_type, openai.api_base, openai.api_version)

embeddings = OpenAIEmbeddings(model='text-embedding-ada-002',
                              deployment='emb0614',
                              openai_api_base=os.getenv("OPENAI_API_BASE"),
                              openai_api_type=os.getenv("OPENAI_API_TYPE"),
                              chunk_size=1)

txt_docsearch = Chroma(persist_directory='apidocs', embedding_function=embeddings)

azure https://test0406.openai.azure.com/ 2023-03-15-preview


In [8]:
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory

llm = ChatOpenAI(engine='test0406', model_name="gpt-3.5-turbo", temperature=0.5, max_tokens=2048)

template_rewrite = """根据聊天记录和用户问题，请把用户问题重写为一个不需要上下文、包含了所有相关信息的独立问题。
聊天记录:
{chat_history}
用户问题: {question}
独立问题:"""

template_refine = """请检查下述检索结果是否包含了完成目标任务的函数信息。
检索结果:
{context_str}

如果这个检索结果包含了有用的函数信息，请根据它和原有的代码重新生成更好的实现。如果没有相关信息，就返回原有的代码。
新的回复："""

template_summary = """根据聊天内容增量式生成摘要，在原始摘要上附加新的摘要。
EXAMPLE
当前摘要: 
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

新的聊天内容:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

新的摘要:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

当前摘要:
{summary}

新的聊天内容:
{new_lines}

新的摘要:
"""

# 初始化 prompt 对象
prompt_rewrite = ChatPromptTemplate.from_template(template_rewrite)
# 初始化问答链
question_generator = LLMChain(llm=llm, prompt=prompt_rewrite)

prompt_refine = ChatPromptTemplate.from_template(template_refine)
doc_chain = load_qa_chain(llm, chain_type="refine", refine_prompt=prompt_refine)

prompt_summary = PromptTemplate(input_variables=["summary", "new_lines"], template=template_summary)
memory = ConversationSummaryBufferMemory(llm=llm, prompt = prompt_summary,
        max_token_limit=1000, memory_key="chat_history", return_messages=True)

chain = ConversationalRetrievalChain(
    retriever=txt_docsearch.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    memory=memory
)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [9]:
query = """操作步骤及说明
1.	业务人员配置好银行账号数据表，和邮箱账号数据；
2.	读取账号配置表，根据账号信息自动登录网银系统；
3.	获取前一天历史余额数据，写入资金日报表模板对应银行账户余额栏；
4.	下载前一天电子回单、网银流水，按照固定规则命名（命名规则：公司_收/付_账号_日期，例：X公司_收_账号_20211222），并在日志记录表中写入记录；
5.	将日报excel及回、流水文件夹进行压缩；
6.	邮件发送运行异常提醒，将运行的数据压缩文件发送到指定邮箱；
7.	业务人员接收邮件文件，进行确认操作。
"""
result = chain({"question": query})
print(result)

{'question': '操作步骤及说明\n1.\t业务人员配置好银行账号数据表，和邮箱账号数据；\n2.\t读取账号配置表，根据账号信息自动登录网银系统；\n3.\t获取前一天历史余额数据，写入资金日报表模板对应银行账户余额栏；\n4.\t下载前一天电子回单、网银流水，按照固定规则命名（命名规则：公司_收/付_账号_日期，例：X公司_收_账号_20211222），并在日志记录表中写入记录；\n5.\t将日报excel及回、流水文件夹进行压缩；\n6.\t邮件发送运行异常提醒，将运行的数据压缩文件发送到指定邮箱；\n7.\t业务人员接收邮件文件，进行确认操作。\n', 'chat_history': [HumanMessage(content='操作步骤及说明\n1.\t业务人员配置好银行账号数据表，和邮箱账号数据；\n2.\t读取账号配置表，根据账号信息自动登录网银系统；\n3.\t获取前一天历史余额数据，写入资金日报表模板对应银行账户余额栏；\n4.\t下载前一天电子回单、网银流水，按照固定规则命名（命名规则：公司_收/付_账号_日期，例：X公司_收_账号_20211222），并在日志记录表中写入记录；\n5.\t将日报excel及回、流水文件夹进行压缩；\n6.\t邮件发送运行异常提醒，将运行的数据压缩文件发送到指定邮箱；\n7.\t业务人员接收邮件文件，进行确认操作。\n', additional_kwargs={}, example=False), AIMessage(content="根据检索结果，可以使用sendCustomHttpMsg函数向iS-RPA企业服务器发送自定义的http消息。下面是一个示例代码：\n\nimport requests\nimport json\nimport time\n\ndef sendCustomHttpMsg(jsonData, address, port, retryTimes=2, timeout=5):\n    url = address + ':' + str(port)\n    headers = {'Content-Type': 'application/json'}\n    for i in range(retryTimes):\n        try:\n            resp

In [10]:
print(result['answer'])

根据检索结果，可以使用sendCustomHttpMsg函数向iS-RPA企业服务器发送自定义的http消息。下面是一个示例代码：

import requests
import json
import time

def sendCustomHttpMsg(jsonData, address, port, retryTimes=2, timeout=5):
    url = address + ':' + str(port)
    headers = {'Content-Type': 'application/json'}
    for i in range(retryTimes):
        try:
            response = requests.post(url, data=json.dumps(jsonData), headers=headers, timeout=timeout)
            if response.status_code == 200:
                return True
        except:
            time.sleep(timeout)
    return False

# 示例用法
jsonData = {'a': 'aaa', 'b': 'bbb'}
address = 'http://192.168.0.184'
port = 8080
retryTimes = 2
timeout = 5
result = sendCustomHttpMsg(jsonData, address, port, retryTimes, timeout)
print(result)

注意，这里的示例代码仅供参考，具体实现可能需要根据实际情况进行调整。
